In [18]:
from os import listdir
from os.path import isfile, join
import random
from PIL import Image

import numpy as np
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
Learning_Rate=1e-5
width=height=900 # image width and height
batchSize=1

In [34]:
torch.cuda.is_available()

True

In [33]:
path = 'GeneratedImages'
files = [f for f in listdir(path) if isfile(join(path, f))]
len(files)

144001

In [35]:
def LoadRandomImg():
    path = 'GeneratedImages/' + files[random.randrange(0,len(files))]
    image = Image.open(path)
    return np.array(image)

In [36]:
def LoadBatch():
    images = torch.zeros([batchSize,3,height,width])
    FillLevel = torch.zeros([batchSize])
    for i in range(batchSize):
        images[i],FillLevel[i]=LoadRandomImg()
    return images,FillLevel

In [ ]:
#--------------Load and set net and optimizer-------------------------------------

# Set device GPU or CPU where the training will take place
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

#load net
Net = torchvision.models.resnet18(pretrained=True)

# Change final layer to predict one value
Net.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True) 

Net = Net.to(device)

# Create adam optimizer
optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) 



#----------------Train------------------------------------------------------------

# Save average loss for display
AverageLoss=np.zeros([50]) 

# Training loop
for itr in range(500001): 
    # Load taining batch
    images,GTFillLevel=LoadBatch()

    # Load image
    images=torch.autograd.Variable(images,requires_grad=False).to(device) 
    
    # Load Ground truth fill level
    GTFillLevel = torch.autograd.Variable(GTFillLevel, requires_grad=False).to(device) 
    
    # make prediction
    PredLevel=Net(images)
    Net.zero_grad()
    Loss=torch.abs(PredLevel-GTFillLevel).mean()
    
    # Backpropogate loss
    Loss.backward() 
    
    # Apply gradient descent change to weight
    optimizer.step() 
    
    # Save loss average
    AverageLoss[itr%50]=Loss.data.cpu().numpy() 
    
    print(itr,") Loss=",Loss.data.cpu().numpy(),'AverageLoss',AverageLoss.mean()) # Display loss
    
    # Save model
    if itr % 200 == 0: 
        #Save model weight
        print("Saving Model" +str(itr) + ".torch") 
        torch.save(Net.state_dict(),   str(itr) + ".torch")